## MC
------------

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import datetime as dt
from stock_forecasting.config import PROCESSED_DATA_DIR

---------------

In [5]:
df = pdr.DataReader('^SPX', data_source='stooq', start='2010-1-1', end='2023-12-31')

In [10]:
def get_data(stocks, start, end):
    # stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    # stockData = stockData['Close']
    stockData = pd.read_pickle(PROCESSED_DATA_DIR / "filtered.pkl")
    returns = stockData.xs("Close", level="Price", axis=1).pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

stockList = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock + '.AX' for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 300)


meanReturns, covMatrix = get_data(stocks, startDate, endDate)

print(meanReturns)

#give random weights to portfolio
weights = np.random.random(len(meanReturns))
weights/=np.sum(weights)

print(weights)



Ticker
HAL     0.000488
K       0.000235
IRM     0.000732
HES     0.000624
KEY     0.000660
          ...   
DLR     0.000604
DD      0.000419
SWK     0.000461
MSFT    0.000940
ROK     0.000787
Length: 392, dtype: float64
[4.29907687e-04 2.42859600e-05 3.99227483e-03 4.45963622e-03
 2.23153802e-03 3.96503643e-03 2.98563020e-03 3.12772607e-03
 4.51964529e-03 3.07299334e-03 7.41654886e-04 3.25195352e-03
 7.52987985e-04 3.72721383e-03 2.57701062e-03 4.13210419e-03
 3.09784910e-04 3.07587513e-03 2.88812138e-03 2.07636028e-03
 2.59809397e-03 1.76411192e-03 2.16175377e-03 1.33602321e-03
 2.68471031e-04 1.38029664e-03 3.77076233e-03 3.19705931e-03
 4.67412394e-04 2.84228508e-03 7.76727386e-04 1.97251689e-04
 9.21672390e-04 3.31801854e-04 2.74033710e-03 3.36295746e-03
 5.73983054e-04 2.64037437e-04 1.31613237e-03 4.70386966e-03
 3.66045628e-03 1.58476134e-03 2.50327507e-04 3.63851655e-04
 1.51695935e-03 1.89318001e-03 2.94898706e-03 1.71959547e-04
 2.17590617e-04 4.79907570e-03 8.13577901e-04 

In [3]:
#MC simulater
#nº of simulations
mc_simes = 100
T = 31 #days

meanM = np.full(shape=(T, len(weights)), fill_value=meanReturns)
meanM = meanM.T #transpose

portfolio_sins = np.full(shape=(T, mc_simes), fill_value=0.0)

initialPortfolio = 10000 #started with initialPortfolio money

for m in range(0, mc_simes):
    #mc loops
    Z = np.random.normal(size=(T, len(weights))) #T com o nº de stocks
    L = np.linalg.cholesky(covMatrix) #lower triangle // nº of stocks by the n of stocks 

    dailyReturns = meanM + np.inner(L, Z)
    portfolio_sins[:, m] = np.cumprod(np.inner(weights, dailyReturns.T)+1)*initialPortfolio

plt.plot(portfolio_sins)
plt.ylabel('Portfolio value ($)')
plt.xlabel('Days')
plt.title('MC simulation of a stock portfolio')
plt.show()


NameError: name 'weights' is not defined

In [ ]:
#value of risk and conditional value of risk

def mcVar(returns, alpha = 5):
    ''' 
    input: pandas series of returns 
    output: percentile on return distribution to a given confidence level alpha
    '''

    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    else:
        raise TypeError("Expected a pandas data series.")


def mcCVar(returns, alpha = 5):
    ''' 
    input: pandas series of returns 
    output: CVar or Expected Shortfall to a given confidence level alpha
    '''

    if isinstance(returns, pd.Series):
        belowVar = returns <= mcVar(returns, alpha=alpha)
        return returns[belowVar].mean()
    else:
        raise TypeError("Expected a pandas data series.")
    

portResults = pd.Series(portfolio_sins[-1,:])

VaR = initialPortfolio - mcVar(portResults, alpha=5)
CVaR = initialPortfolio - mcCVar(portResults, alpha=5)

print(f'Var ${format(round(VaR, 2))}')  #limite (o valor esperado a perder quando fazer inicial - variancia)
print(f'Var ${format(round(CVaR, 2))}') #risco conditional risc (para baixo do valor obtido em cima)

> next part is an example

In [4]:
# functions from the pandas library to record historical index prices from the Stooq platform; the start 
df = wb.DataReader('^SPX', data_source='stooq', start='2010-1-1', end='2021-3-11')

In [5]:
# calculation of logarithmic rates of return, average rate of return and standard deviation and drift
logarithmic_returns = np.log(1 + df['Close'].pct_change())
mu_r = logarithmic_returns.mean()
std_r = logarithmic_returns.std()
drift = mu_r - 0.5*(std_r**2)
wyniki = []

In [ ]:
# function that generates 20,000 rates of return consecutive trading days according to Monte Carlo simulation
for i in range(20000):
    d_returns = np.exp(drift + std_r * norm.ppf(np.random.rand(252)))
    list_prices = [df['Close'][0]]
    # function that calculates the new price after taking into account the generated rate of return from the simulation
    for x in d_returns:
        list_prices.append(list_prices[-1]*x)
    plt.plot(list_prices)
    wyniki.append(list_prices)
    lista=[]
    # A loop that adds the results to the time series to be visualized
    for z in results:
        lista.append(z[-1])

C:\Users\Daniela\AppData\Local\Temp\ipykernel_14472\2799356803.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  list_prices = [df['Close'][0]]


KeyboardInterrupt: 

-----

In [ ]:
"""
Compute the logarithmic returns of Google stock
"""
log_return = np.log(1 + data.pct_change())
#Plot
sns.distplot(log_returns.iloc[1:])
plt.xlabel("Daily Return")
plt.ylabel("Frequency")

logarithmic_returns = np.log(1 + df['Close'].pct_change())
mu_r = logarithmic_returns.mean()
std_r = logarithmic_returns.std()
drift = mu_r - 0.5*(std_r**2)

In [ ]:
#compute drift
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5*var)

In [ ]:
'''
Compute the Variance and Daily Returns
In this step we have to generate random variables for every day 
forecasted and for every simulation trial we will run

31 dias->Janeiro(valor de previsão final) 
'''
stdev = log_returns.std()
days = 31
trials = 10000
Z = norm.ppf(np.random.rand(days, trials)) #days, trials
daily_returns = np.exp(drift.values + stdev.values * Z)

In [ ]:
''' 
Calculating the stock price for every trial
'''
price_paths = np.zeros_like(daily_returns)
price_paths[0] = data.iloc[-1]
for t in range(1, days):
    price_paths[t] = price_paths[t-1]*daily_returns[t]

''' 
With the price_paths matrix, now you can calculate the probability of profitability, 
or the expected annualized returm, for example. 
'''

In [ ]:
#fazer func
def MC(df, predict_date:31, n_diff_predicts):
    #P1 -> Create simulations to explore a variety of portfolio allocations
    #get returns of stocks
    #fazer log
    log_returns = np.log(1 + df['Close'].pct_change())
    #obter mean and var
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5*var)

    #deviation between the two, try to understand

    #P2 -> identify the one that optimizes your objective (e.g., maximize return, minimize risk, or optimize the Sharpe ratio)
    


    ...

------------
Automating the Monte Carlo Simulations and Showing CAPM Metrics
------------

----------
References

> https://medium.com/analytics-vidhya/monte-carlo-simulations-for-predicting-stock-prices-python-a64f53585662
> https://journals.ue.wroc.pl/fins/article/download/138/7/458
>https://youtu.be/f9MAFvP5-pA